In [2]:
import carla
import random
import time
import os
import cv2
import numpy as np
from queue import Queue

# Connect to the CARLA server
def connect_to_carla():
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    world = client.get_world()
    return client, world

# Generate dataset with different weather conditions
def generate_dataset(world, output_dir='dataset', num_samples=100):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Define weather conditions
    weather_conditions = {
        'day': carla.WeatherParameters.ClearNoon,
        'night': carla.WeatherParameters.ClearNight,
        'rain': carla.WeatherParameters.HardRainNoon,
        'fog': carla.WeatherParameters(fog_density=80.0)  
    }
    
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('model3')[0]
    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '224')
    camera_bp.set_attribute('image_size_y', '224')
    
    spawn_points = world.get_map().get_spawn_points()
    
    for condition_name, weather in weather_conditions.items():
        world.set_weather(weather)
        condition_dir = os.path.join(output_dir, condition_name)
        os.makedirs(condition_dir, exist_ok=True)
        
        for i in range(num_samples):
            # Select a random spawn point
            spawn_point = random.choice(spawn_points)
            # Spawning the vehicle
            vehicle = world.spawn_actor(vehicle_bp, spawn_point)
            # Attaching the camera to the vehicle
            camera = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=2.5, z=0.7)), 
                                       attach_to=vehicle)
            
            image_queue = Queue()
            camera.listen(lambda image: image_queue.put(image))
            
            # Take a step in the simulation
            world.tick()
            # Capturing the image
            image = image_queue.get()
            image.save_to_disk(os.path.join(condition_dir, f'{i}.png'))
            
            # Cleanup
            camera.stop()
            vehicle.destroy()
            time.sleep(0.1)  # A small delay for stability
        
    print("Dataset generation completed!")

def main():
    client, world = connect_to_carla()
    generate_dataset(world)

if __name__ == '__main__':
    main()

Dataset generation completed!
